<a href="https://colab.research.google.com/github/SaiMaster-12/Pothole-Detection-System/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving best.pt to best.pt


In [ ]:
# importing drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import os
print(os.path.exists('/content/drive/My Drive/Sai/Project/drive/in'))

True


In [ ]:
!pip install jedi>=0.16

In [ ]:
!pip install --upgrade pip setuptools wheel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.7 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.1.0
    Uninstalling setuptools-75.1.0:
      Successfully uninstalled setuptools-75.1.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [ ]:
!pip install ultralytics==8.0.38

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.1/278.1 kB 8.9 MB/s eta 0:00:00


In [ ]:
# No. of files
import os

# Specify the path
path = r"/content/drive/My Drive/Sai/Project/drive/Output/test/images"

# List all files and directories in the specified path
files = os.listdir(path)

# Define a list of valid image file extensions
image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff', '.webp', '.txt']

# Count only image files
image_count = sum(1 for file in files if os.path.isfile(os.path.join(path, file)) and file.lower().endswith(tuple(image_extensions)))

print(f"Number of image files in the directory: {image_count}")

Number of image files in the directory: 81


In [ ]:
# Spliting
import os
import shutil

# Define paths
images_path = r"/content/drive/My Drive/Sai/Project/drive/in"
annotations_path = r"/content/drive/My Drive/Sai/Project/drive/Anoo"
output_dir = r"/content/drive/My Drive/Sai/Project/drive/Output"

# Output folders
train_images = os.path.join(output_dir, "train", "images")
train_annotations = os.path.join(output_dir, "train", "labels")
val_images = os.path.join(output_dir, "val", "images")
val_annotations = os.path.join(output_dir, "val", "labels")
test_images = os.path.join(output_dir, "test", "images")
test_annotations = os.path.join(output_dir, "test", "labels")

# Split ratios
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

# Create directories
for path in [train_images, train_annotations, val_images, val_annotations, test_images, test_annotations]:
    os.makedirs(path, exist_ok=True)

# Get all images and corresponding annotations
images = [f for f in os.listdir(images_path) if f.endswith(('.jpg', '.jpeg', '.png'))]
images_with_annotations = [
    img for img in images if os.path.exists(os.path.join(annotations_path, f"{os.path.splitext(img)[0]}.txt"))
]

# Sort images to maintain reproducibility
images_with_annotations.sort()

# Calculate splits
total_images = len(images_with_annotations)
train_split = int(total_images * train_ratio)
val_split = train_split + int(total_images * val_ratio)

train_files = images_with_annotations[:train_split]
val_files = images_with_annotations[train_split:val_split]
test_files = images_with_annotations[val_split:]

# Function to move files
def move_files(file_list, dest_images, dest_annotations):
    for file_name in file_list:
        base_name = os.path.splitext(file_name)[0]
        image_src = os.path.join(images_path, file_name)
        annotation_src = os.path.join(annotations_path, f"{base_name}.txt")

        shutil.copy(image_src, dest_images)
        shutil.copy(annotation_src, dest_annotations)

# Move files
move_files(train_files, train_images, train_annotations)
move_files(val_files, val_images, val_annotations)
move_files(test_files, test_images, test_annotations)

print("Data split complete!")


Data split complete!


In [ ]:
# Training, Validation, Testing 👇🏻

In [ ]:
from ultralytics import YOLO

# Paths
data_yaml = "/content/data.yaml"  # Path to the data.yaml file
pretrained_model = "/content/best.pt"  # Path to the pretrained model
test_video = "/content/drive/My Drive/Sai/Project/vid/vid_list/test_video.mp4"
output_video = "/content/drive/My Drive/Sai/Project/vid/Outputs/test_video_output.mp4"
output_dir = "/content/drive/My Drive/Sai/Project/vid/Outputs"

# Load the YOLOv8 model
model = YOLO(pretrained_model)

# Training the model
print("Starting training...")
model.train(
    data=data_yaml,           # Dataset configuration file
    epochs=50,                # Number of training epochs
    imgsz=640,                # Image size for training
    batch=16,                 # Batch size
    project=output_dir,       # Project directory
    name="fine_tune_yolo",    # Name of the training run
    pretrained=True           # Use pretrained weights
)

# Validate the model
print("Validating the model...")
metrics = model.val(data=data_yaml)  # Perform validation
print(f"Validation metrics: {metrics}")

# Test the model on a new video
print("Testing the model on a new video...")
results = model.predict(
    source=test_video,        # Path to the test video
    save=True,                # Save output predictions
    project=output_dir,       # Directory for saving results
    name="video_results"      # Name of the result directory
)

print(f"Testing complete. Output video saved at: {output_video}")


/usr/local/lib/python3.10/dist-packages/ultralytics/nn/tasks.py:338: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location='cpu')  # load
Ultral

Starting training...


Model summary: 225 layers, 3011043 parameters, 3011027 gradients, 8.2 GFLOPs

Transferred 355/355 items from pretrained weights
optimizer: SGD(lr=0.01) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.001), 63 bias
train: Scanning /content/drive/My Drive/Sai/Project/vid/vid_test_frames/train/labels.cache... 192 images, 81 backgrounds, 0 corrupt: 100%|██████████| 192/192 [00:00<?, ?it/s]
/usr/local/lib/python3.10/dist-packages/albumentations/core/composition.py:205: UserWarning: Got processor for bboxes, but no transform to process it.
  self._set_keys()
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Scanning /content/drive/My Drive/Sai/Project/vid/vid_test_frames/valid/labels.cache... 35 images, 29 backgrounds, 0 corrupt: 100%|██████████| 35/35 [00:00<?, ?it/s]
Image sizes 640 train, 640 val
Using 2 data

Validating the model...


val: Scanning /content/drive/My Drive/Sai/Project/vid/vid_test_frames/valid/labels.cache... 35 images, 29 backgrounds, 0 corrupt: 100%|██████████| 35/35 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.31it/s]
                   all         35         14        0.7      0.357       0.39      0.179
Speed: 0.4ms pre-process, 5.4ms inference, 0.0ms loss, 3.7ms post-process per image
Ultralytics YOLOv8.0.38 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)



Validation metrics: <ultralytics.yolo.utils.metrics.DetMetrics object at 0x7a8bc04e6b00>
Testing the model on a new video...


video 1/1 (1/2625) /content/drive/My Drive/Sai/Project/vid/vid_list/test_video.mp4: 640x384 76.1ms
video 1/1 (2/2625) /content/drive/My Drive/Sai/Project/vid/vid_list/test_video.mp4: 640x384 14.9ms
video 1/1 (3/2625) /content/drive/My Drive/Sai/Project/vid/vid_list/test_video.mp4: 640x384 8.2ms
video 1/1 (4/2625) /content/drive/My Drive/Sai/Project/vid/vid_list/test_video.mp4: 640x384 7.5ms
video 1/1 (5/2625) /content/drive/My Drive/Sai/Project/vid/vid_list/test_video.mp4: 640x384 7.7ms
video 1/1 (6/2625) /content/drive/My Drive/Sai/Project/vid/vid_list/test_video.mp4: 640x384 7.9ms
video 1/1 (7/2625) /content/drive/My Drive/Sai/Project/vid/vid_list/test_video.mp4: 640x384 7.3ms
video 1/1 (8/2625) /content/drive/My Drive/Sai/Project/vid/vid_list/test_video.mp4: 640x384 7.3ms
video 1/1 (9/2625) /content/drive/My Drive/Sai/Project/vid/vid_list/test_video.mp4: 640x384 12.5ms
video 1/1 (10/2625) /content/drive/My Drive/Sai/Project/vid/vid_list/test_video.mp4: 640x384 8.0ms
video 1/1 (11/26

Testing complete. Output video saved at: /content/drive/My Drive/Sai/Project/vid/Outputs/test_video_output.mp4


In [ ]:
# Tr, Va, Te with already exixting model

In [ ]:
# Training, Validation, Testing

from ultralytics import YOLO
import shutil  # For moving the saved model

# Paths
data_config_path = "/content/data (1).yaml"  # YAML file uploaded earlier
model_save_path = "/content/drive/My Drive/Sai/Project/drive/PotholeDetectionModel.pt"
test_images_path = "/content/drive/My Drive/Sai/Project/drive/Output/test/images"
predictions_output_path = "/content/drive/My Drive/Sai/Project/drive/Predictions"

# Load the pre-trained YOLOv8 model
model = YOLO('yolov8n.pt')  # Use 'yolov8n.pt' for a lightweight model

# Train the model
model.train(
    data=data_config_path,  # Use the YAML file for dataset configuration
    epochs=50,              # Adjust based on your needs
    imgsz=640,              # Input image size
    batch=16,               # Adjust based on available GPU memory
    save=True,              # Save training checkpoints
    name="pothole-detection"
)

# Validate the model
print("Validating the model...")
validation_results = model.val(data=data_config_path)
print("Validation Results:", validation_results)

# Test the model
print("Testing the model...")
test_results = model.predict(
    source=test_images_path,   # Path to the test images
    conf=0.25,                 # Confidence threshold
    save=True,                 # Save predictions with bounding boxes
    save_txt=True,             # Save predictions as text files
    project=predictions_output_path,  # Output directory for predictions
    name="TestResults"
)
print("Test Results:", test_results)

# Export the trained model
print("Exporting the trained model...")
model.export(format='torchscript')  # Export the model in TorchScript format

# Move the saved model file to the desired location
trained_model_path = f"runs/train/pothole-detection/weights/best.pt"  # Default save location
shutil.copy(trained_model_path, model_save_path)
print(f"Trained model moved to {model_save_path}")

In [ ]:
# Only testing with image

In [ ]:
# Test the model
from ultralytics import YOLO
import shutil  # For moving the saved model

# Paths
pretrained_model = "/content/best.pt"  # Path to the pretrained model
test_video = "/content/drive/My Drive/Sai/Project/vid/vid_list/test_video1.mp4"
output_video = "/content/drive/My Drive/Sai/Project/vid/Outputs/test_video_output1.mp4"
output_dir = "/content/drive/My Drive/Sai/Project/vid/Outputs"

# Load the YOLOv8 model
model = YOLO(pretrained_model) # Change this to a offline loading method.

print("Testing the model...")
test_results = model.predict(
    source=test_images_path,   # Path to the test images
    conf=0.25,                 # Confidence threshold
    save=True,                 # Save predictions with bounding boxes
    save_txt=True,             # Save predictions as text files
    project=predictions_output_path,  # Output directory for predictions
    name="TestResults"
)
print("Test Results:", test_results)


In [ ]:
# Only Testing with video

from ultralytics import YOLO

# Paths
#data_yaml = "/content/data.yaml"  # Path to the data.yaml file
pretrained_model = "/content/best.pt"  # Path to the pretrained model
test_video = "/content/drive/My Drive/Sai/Project/vid/vid_list/test_video.mp4"
output_video = "/content/drive/My Drive/Sai/Project/vid/Outputs/test_video_output1.mp4"
output_dir = "/content/drive/My Drive/Sai/Project/vid/Outputs"

# Load the YOLOv8 model
model = YOLO(pretrained_model) # Change this to a offline loading method.

# Test the model on a new video
print("Testing the model on a new video...")
results = model.predict(
    source=test_video,        # Path to the test video
    save=True,                # Save output predictions
    project=output_dir,       # Directory for saving results
    name="video_results"      # Name of the result directory
)

print(f"Testing complete. Output video saved at: {output_video}")


In [ ]:
# If you want to change the directory

In [ ]:
import os
import shutil

# Source and destination directories
source_dir = "/content/runs/detect_video"
destination_dir = "/content/drive/My Drive/Sai/Project/drive/Predictions/Video_tester"

# Ensure the destination directory exists
os.makedirs(destination_dir, exist_ok=True)

# Move files
for file_name in os.listdir(source_dir):
    source_file = os.path.join(source_dir, file_name)
    destination_file = os.path.join(destination_dir, file_name)

    # Move the file
    shutil.move(source_file, destination_file)
    print(f"Moved: {source_file} -> {destination_file}")

print("All files moved successfully!")

Moved: /content/runs/detect_video/input_01.mp4 -> /content/drive/My Drive/Sai/Project/drive/Predictions/Video_tester/input_01.mp4
All files moved successfully!


In [ ]:
# Testing Mail (Only mail)

In [1]:
!pip install sendgrid

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.1/102.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 3.6 MB/s eta 0:00:00


In [ ]:
# Mail
import os
import base64  # For base64 encoding
from sendgrid import SendGridAPIClient
from sendgrid.helpers.mail import Mail, Attachment, FileContent, FileName, FileType, Disposition

# Your SendGrid API Key
SENDGRID_API_KEY = ""

# Function to send an email with attachments
def send_email(location, files):
    # Create the email message
    message = Mail(
        from_email="3br21ai035@bitm.edu.in",  # Replace with your verified SendGrid sender email
        to_emails="saikiranjavalkar@gmail.com",  # Recipient's email
        subject="Pothole Detected",
        html_content=f"""
        <p><b>Pothole Detected!</b></p>
        <p>Location details:</p>
        <ul>
            <li><b>Latitude:</b> {location[0]}</li>
            <li><b>Longitude:</b> {location[1]}</li>
        </ul>
        """
    )

    # Attach files to the email
    for file_path in files:
        with open(file_path, 'rb') as f:
            # Read the file as bytes
            file_data = f.read()
            # Base64 encode the file data
            encoded_file_data = base64.b64encode(file_data).decode()  # Base64 encode and convert to string

            file_name = os.path.basename(file_path)

            # Create the attachment
            encoded_file = Attachment(
                FileContent(encoded_file_data),  # Base64 encoded content as string
                FileName(file_name),
                FileType("image/jpeg"),  # Assuming jpeg image
                Disposition("attachment")
            )

            # Add the attachment to the message
            message.add_attachment(encoded_file)

    # Send the email
    try:
        sg = SendGridAPIClient(SENDGRID_API_KEY)
        response = sg.send(message)
        print(f"Email sent! Status Code: {response.status_code}")
    except Exception as e:
        print(f"Error sending email: {e}")

# Example usage
if __name__ == "__main__":
    location = (12.9716, 77.5946)  # Example location (latitude, longitude)
    files = ["/content/drive/My Drive/Sai/Project/drive/Output/test/images/ph_0_5362.jpeg"]  # Replace with actual file paths
    send_email(location, files)


Email sent! Status Code: 202
